# <center> neo4j Ingestion Template </center>

## Notebook setup

In [1]:
!pip install pyneoinstance boto3 --quiet

In [2]:
import os
import re
import string
import warnings
import pandas as pd
import numpy as np
from typing import Tuple
from pyneoinstance import Neo4jInstance
from pyneoinstance.fileload import load_yaml_file

warnings.filterwarnings('ignore')

### Naming Conventions & Recommendations

In [3]:
# Follow these best practices 
# https://neo4j.com/docs/cypher-manual/current/syntax/naming/

## Utility Functions

In [4]:
def get_parallel_info(num_rows: int) -> Tuple[int,bool]:
    partitions = 1
    parallel = True
    batches = int(num_rows/1000)
    if batches > 1:
        partitions = batches
        parallel = True
    return (partitions, parallel)

In [38]:
config_file = load_yaml_file('cypher.yaml')
neo_info = config_file['neo4j']
pre_load_queries = config_file['queries']['pre-load']
node_queries = config_file['queries']['load']['nodes']
rela_queries = config_file['queries']['load']['relationships']
#post_load_queries = config_file['queries']['post-load']
node_labels = list(node_queries.keys())
rela_types = list(rela_queries.keys())

In [8]:
filtered = df_claim_nts[df_claim_nts['iEntityType'].notnull()]
filtered.head()

,iId,iClaimId,sNoteType,sText,bSystemNote,iInsertedUserId,dtInsertedTime,iUpdatedUserId,dtUpdatedTime,sEnteredByUserName,sUpdatedByUserName,dtReminderActive,dtReminderIssued,sReminderSentBy,iExpectedOdom,iEntityType,iAssignedToUserId,bExternal
939390,27390558,4908272,0,"CCC for Failure#1:\nComplaint:\nCS every time the vehicle is started there is a buzzing noise that goes away after he reaches 25 MPH\n\nCause:\nthe buzzing noise is coming from the water pump, it is leaking from the gasket, coolant level was at the max and in looked like it could be changed, NO signs of damage, rust or corrosion, mods, or commercial use\n\nCorrection:\nReplace water pump\n\nCCC for Failure#2:\nComplaint:\nTF\n\nCause:\nboth outer tie rod ends have play and the boots are split, NO signs of damage\n\nCorrection:\nReplace both outer tie rods\n\nCCC for Failure#3:\nComplaint:\nTF\n\nCause:\ncompressor is not engaging, does have good power and ground going to it, freon was full, no signs of damage or rust or corrosion\n\nCorrection:\nReplace a/c compressor\n\nCCC for Failure#4:\nComplaint:\nTF\n\nCause:\nfound 2 TPM sensor have failed, both the rear sensors are shorted, NO signs of damage\n\nCorrection:\nReplace 2 TPM sensors\n\nCCC for Failure#5:\nComplaint:\nTF\n\nCause:\ndrive belt is frayed/worn\n\nCorrection:\nReplace drive belt\n\n",True,83141.00,2022-04-02 12:10:20.437000,83141,2022-04-02 12:10:20.437000,ALEX VALLES,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939391,27390559,4908272,0,"New Claim Created. Loss Odometer: 96,051, Loss Date: 04/02/2022",True,83141.00,2022-04-02 12:10:20.437000,83141,2022-04-02 12:10:20.437000,ALEX VALLES,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939392,27390568,4907723,0,shop cb - states ch states we are now covering claim = sent message to james d for view,False,0.00,2022-04-02 12:14:10.537000,0,2022-04-02 12:14:10.537000,Dean Melchiorre,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939393,27390576,4908226,0,*** CLAIM RESOLUTION EVENT ***\n \nTOTAL PAYABLE TO RF:\n0.00\n \nLINES AUTHORIZED:\nn/a\n \nLINES DENIED:\nno1-2\n \nREASON FOR DENIAL:\npart not listed for coverage,False,0.00,2022-04-02 12:16:50.797000,0,2022-04-02 12:16:50.797000,Shaun Ivory,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939394,27390577,4908226,0,CCC for Failure#1:\nComplaint:\nAdvised us that he needed an alternator.\n\nCause:\nBattery failed load test.\nP0031\nAF Ratio sensor not fluctuating\npower and ground good to sensor\n\nCorrection:\nReplace Battery. Alternator needs to be re-checked once battery is replaced.\nAF Ratio\n\nCCC for Failure#2:\nComplaint:\ninspection\n\nCause:\nps belt worn out\nbelt tensioner making noise\nidler pulley making noise\n\nCorrection:\ndrive belt tensioner\nidler pulley\n\n,True,78905.00,2022-04-02 12:16:56.870000,78905,2022-04-02 12:16:56.870000,Shaun Ivory,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN


In [9]:
df_claim_nts.head()

,iId,iClaimId,sNoteType,sText,bSystemNote,iInsertedUserId,dtInsertedTime,iUpdatedUserId,dtUpdatedTime,sEnteredByUserName,sUpdatedByUserName,dtReminderActive,dtReminderIssued,sReminderSentBy,iExpectedOdom,iEntityType,iAssignedToUserId,bExternal
0,22470440,4247341,0,Claim status is changed from Open To Paid.,False,0.00,2019-07-05 07:13:00,0,2020-08-30 16:43:08.420000,BATCHIMPORT,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
1,22470441,4254429,0,The inspection request has been received Request ID 2739962.,False,0.00,2018-10-05 10:17:00,0,2020-08-30 16:43:08.420000,scott.davis,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
2,22470442,4253399,0,Claim is created.,False,0.00,2018-01-05 11:04:00,0,2020-08-30 16:43:08.420000,robert.zadlo,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
3,22470443,4257884,0,Awaiting AmTrust email HUD David P,False,0.00,2017-09-27 10:14:00,0,2020-08-30 16:43:08.420000,monique.rivera,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
4,22470444,4246217,0,"I past history note stated :Reviewed start mileage with Rob V, Ok to proceed with claim . from 07/03/18",False,0.00,2018-11-07 11:00:00,0,2020-08-30 16:43:08.420000,giovanni.fioccola,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN


In [10]:
 df_claim_nts['iInsertedUserId'].fillna(value=-1, inplace=True)
 df_claim_nts['iAssignedToUserId'].fillna(value=-1, inplace=True)

In [11]:
df_claim_nts['iInsertedUserId'] = df_claim_nts['iInsertedUserId'].astype(int)
df_claim_nts['iAssignedToUserId'] = df_claim_nts['iAssignedToUserId'].astype(int)

In [12]:
df_claim_hd.head()

,iId,iContractId,sClaimNumber,sRONumber,sClaimStatusCode,sClaimStatusDesc,dRODate,dClaimDate,iServiceOdom,sPriorityCode,sActivityCode,nActualCashValue,iAssignedToId,sAssignedToFirstName,sAssignedToLastName,dtInserted,dtUpdated,iInsertedUserId,iUpdatedUserId,btCommunicationPreferenceSpecialHandling,iSpecialHandlingServicerId,sSpecialHandlingEntityType,dtEstimatedCompletionDate,sClaimRefNumber
0,4205674,13898388,E_C000009290,NaN,D,Denied,2015-06-03,2015-06-03,96080,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
1,4205675,13871780,E_C000022571,57821,X,Paid,2016-08-22,2016-08-22,87393,NaN,103,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
2,4205676,13871780,E_C000018718,55873,X,Paid,2016-05-23,2016-05-23,73666,NaN,113,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
3,4205677,13871851,E_C000024345,405764,W,Closed,2016-10-03,2016-10-03,39499,NaN,103,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
4,4205678,13871839,E_C000024106,74169,X,Paid,2016-09-27,2016-09-27,51410,NaN,103,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
filtered = df_claim_det[df_claim_det['iClaimPayeeId'] == 4366220]
filtered

,iId,iClaimId,iClaimCompontentId,iClaimPayeeId,sDetailStatusCode,sDetailStatusDesc,sDetailTypeCode,iStatusReasonId,sReasonCode,sReasonDescription,sDetailTypeDescription,iLossCodeId,sLossCode,sLossCodeDescription,bSubletFlag,btGoodWill,iWarrantyMonth,iWarrantyOdom,sPartNumber,sPartDescription,nPartReqQty,nPartReqUnitCost,nPartAuthQty,nPartAuthUnitCost,iTireTread,nPartSubmitQty,cPartSubmitTotal,nLaborSubmitQty,cLaborSubmitTotal,cPartTax1,cPartTax2,cLaborTax1,cLaborTax2,nLaborReqQty,nLaborReqUnitCost,nLaborAuthQty,nLaborAuthUnitCost,sPayMethod,dAuthorizedDate,sAuthNumber,dDeniedDate,dApprovedToPayDate,dtPaymentProcessed,cTotalPending,cTotalDenied,cTotalPaid,sPaymentInstructCode,dtInserted,dtUpdated,bSelfAuthorized,iInsertedUserId,iUpdatedUserId,iRefPaymentId,sLaborOpCode,iAuthorizedUserId,iDeniedUserId
135902,10709622,4205675,5648324,4366220,X,Paid,D,NaN,NaN,NaN,Deductible,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,-100.00,0.00,0.00,WEX,NaN,119706.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,0.00,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN
159398,10733118,4205675,5648324,4366220,X,Paid,J,NaN,NaN,NaN,Job,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,484701ma0b,NaN,1.00,203.95,1.00,164.11,0,1.00,164.11,0.00,0.00,9.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,WEX,NaN,119713.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,173.96,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN
159399,10733119,4205675,5648324,4366220,X,Paid,J,NaN,NaN,NaN,Job,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,0,0.00,0.00,3.00,345.00,0.00,0.00,20.70,0.00,3.00,115.00,3.00,115.00,WEX,NaN,119715.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,365.70,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN
160072,10733792,4205675,5648324,4366220,X,Paid,J,NaN,NaN,NaN,Job,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,488101ma6d,NaN,1.00,2626.45,1.00,2624.45,0,1.00,2624.45,0.00,0.00,157.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00,WEX,NaN,119714.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,2781.92,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN


#### Customer

In [14]:
df_claim_pay.head()

,iId,iClaimId,bDefaultPayee,bDefaultServicer,sPayeeType,sPayeeNumber,sPayeeName,sPayeeContactFirstName,sPayeeContactLastName,sPayeeAddress1,sPayeeAddress2,sPayeeCity,sPayeeState,sPayeeZipCode,sPayeePhoneNumber,sPayeeEmail,sPayMethod,sCreditCardNotifyMethod,cLaborRate,nPartTax1,nPartTax2,nLaborTax1,nLaborTax2,dtInserted,dtUpdated,iInsertedUserId,iUpdatedUserId
0,4249148,4295624,False,False,Servicer,CENTRO,Centro Inspection Agency,NaN,NaN,P.O. Box 309,NaN,Lincroft,NJ,07738,8004773607.00,jaroschak@centroinspection.com,CHK,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
1,4249149,4282403,True,True,Servicer,SC010663,Bryan Honda,NaN,NaN,4104 Raeford Rd,NaN,Fayetville,NC,28309,9104831234.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
2,4249150,4234318,True,True,Servicer,SC002279,Momentum VW,NaN,NaN,19550 NW Freeway,NaN,Jersey Village,TX,77065,2819255100.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
3,4249151,4207583,True,True,Seller,262,Napleton's Mid Rivers CDJR,NaN,NaN,4951 Veterans Memorial Pkwy,NaN,Saint Peters,MO,63376,6369288000.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
4,4249152,4285530,False,False,Servicer,CENTRO,Centro Inspection Agency,NaN,NaN,P.O. Box 309,NaN,Lincroft,NJ,07738,8004773607.00,jaroschak@centroinspection.com,CHK,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN


## Data Load

In [34]:
graph = Neo4jInstance(neo_info['url'],neo_info['user'],neo_info['password'])

### Graph constraints

In [ ]:
# Create indexes and constraints BEFORE loading the data

In [ ]:
graph.execute_write_queries(pre_load_queries,database=neo_info['database'])

### Node Dataframes

In [ ]:
# all dataframes need to be prepared with all the properties the node needs
# the constraints will take care of avoiding duplicate data being loaded, but clean your dataframe as much as possible

In [39]:
nodes_data = {}

#### Address

In [78]:

nodes_data[node_labels[0]]  = df_address

In [67]:
nodes_data[node_labels[7]]  = df_coverage

In [186]:
nodes_data[node_labels[1]]  = df_customers

In [76]:
nodes_data[node_labels[2]]  = df_emails

### Create Nodes

In [ ]:
for node in nodes_data:
    rows = nodes_data[node].shape[0]
    if rows > 0:
        partitions, parallel = get_parallel_info(rows)
        print(f'Loading node {node} with {rows} records')
        result = graph.execute_write_query_with_data(node_queries[node],nodes_data[node],
                                                 database=neo_info['database'],partitions=partitions,
                                                 parallel=parallel, workers=2
                                                )
        print(f'Sucessfully loaded node {node} with the following results: {result}')

In [ ]:
# If you need to create a single node

nodes = ['Address']
for node in nodes:
    rows = nodes_data[node].shape[0]
    partitions, parallel = get_parallel_info(rows)
    print(f'Loading node {node} with {rows} records')
    result = graph.execute_write_query_with_data(node_queries[node],nodes_data[node],
                                                         database=neo_info['database'],
                                                         partitions=partitions,
                                                         parallel=parallel, workers=1
                                                        )
    print(f'Sucessfully loaded {node} with the following results: {result}')

#### Node Quality Test

In [ ]:
nodes_data_size = []
for label, data in nodes_data.items():
    nodes_data_size.append({'nodeLabel':label,'number_rows':data.shape[0]})
nodes_to_load = pd.DataFrame(nodes_data_size)
nodes_loaded = graph.get_node_label_freq()
nodes_qa = pd.merge(nodes_to_load,nodes_loaded,how='left',on='nodeLabel')
nodes_qa['diff'] = nodes_qa['number_rows'] - nodes_qa['frequency']
nodes_qa.sort_values(['frequency'],ascending=False)

### Relationship Dataframes

In [1]:
# The relationship dataframes will consist of the source and target id propety, plus any other value
# you want to set as a property in the relationship

In [ ]:
rela_types

In [40]:
relationships_data = {}

#### HAS_EMAIL

In [290]:
relationships_data[rela_types[1]]  = df_cust_email

### Create Relationships

In [ ]:
for rela in relationships_data:
    rows = relationships_data[rela].shape[0]
    if rows > 0:
        partitions, parallel = get_parallel_info(rows)
        print(f'Loading relationship {rela} with {rows} rows and {partitions} partitions')
        result = graph.execute_write_query_with_data(rela_queries[rela],relationships_data[rela],
                                                 database=neo_info['database'],partitions=partitions,
                                                 parallel=parallel,
                                                 workers=1
                                                )
        print(f'Sucessfully loaded relationship {rela} with the following results: {result}  \n')

In [ ]:
# If you need to load a single relationship

rel = 'CUSTOMER_HAS_ADDRESS'
rows = relationships_data[rel].shape[0]
partitions, parallel = get_parallel_info(rows)
print(f'Loading relationship {rel} with {rows} rows and {partitions} partitions')
result = graph.execute_write_query_with_data(rela_queries[rel],
                                             relationships_data[rel],
                                                 database=neo_info['database'],
                                             partitions=partitions,
                                                 parallel=parallel,
                                             workers=1
                                                )
print(f'Sucessfully loaded relationship {rel} with the following results: {result}')